In [9]:
!pip install pyspark==3.1.2
!pip install delta-spark

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrad

In [3]:
# install jdk 8 which is compatible with pyspark2.4
# please ignore the java-common dependency issue which does not affect the jdk function.
!test -f amazon-corretto-8* || (wget https://corretto.aws/downloads/latest/amazon-corretto-8-x64-linux-jdk.deb && dpkg --install amazon-corretto-8-x64-linux-jdk.deb)

# add hostname into /etc/hosts 
!grep `hostname` /etc/hosts >/dev/null || echo 127.0.0.1 `hostname` >> /etc/hosts

/bin/sh: 1: test: amazon-corretto-8-x64-linux-jdk.deb: unexpected operator
--2021-11-20 00:20:41--  https://corretto.aws/downloads/latest/amazon-corretto-8-x64-linux-jdk.deb
Resolving corretto.aws (corretto.aws)... 13.32.182.20, 13.32.182.60, 13.32.182.29, ...
Connecting to corretto.aws (corretto.aws)|13.32.182.20|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: /downloads/resources/8.312.07.1/java-1.8.0-amazon-corretto-jdk_8.312.07-1_amd64.deb [following]
--2021-11-20 00:20:41--  https://corretto.aws/downloads/resources/8.312.07.1/java-1.8.0-amazon-corretto-jdk_8.312.07-1_amd64.deb
Reusing existing connection to corretto.aws:443.
HTTP request sent, awaiting response... 200 OK
Length: 117954710 (112M) [binary/octet-stream]
Saving to: ‘amazon-corretto-8-x64-linux-jdk.deb.3’

amazon-corretto-8-x 100%[===================>] 112.49M  44.8MB/s    in 2.5s    

2021-11-20 00:20:44 (44.8 MB/s) - ‘amazon-corretto-8-x64-linux-jdk.deb.3’ saved [117954710/1

In [4]:
# specify jdk 
%set_env JAVA_HOME=/usr/lib/jvm/java-1.8.0-amazon-corretto

env: JAVA_HOME=/usr/lib/jvm/java-1.8.0-amazon-corretto


In [52]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DeltaSparkApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "libs/jets3t-0.9.4.jar, libs/hadoop-aws-2.7.3.jar")
spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [46]:
spark

In [7]:
%%bash

mkdir -p Data/delta_demo/loans/ 

wget -O Data/delta_demo/loans/SAISEU19-loan-risks.snappy.parquet https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet 
ls -al Data/

total 76
drwxr-xr-x 4 root root  6144 Nov 19 18:00 .
drwxr-xr-x 5 root root  6144 Nov 20 00:20 ..
drwxr-xr-x 2 root root  6144 Nov 19 18:00 .ipynb_checkpoints
drwxr-xr-x 4 root root  6144 Nov 19 18:01 delta_demo
-rw-r--r-- 1 root root 61065 Oct 11 15:32 tech_ticker_volume.csv


--2021-11-20 00:21:05--  https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.70.206, 104.17.72.206, 104.17.71.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.70.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘Data/delta_demo/loans/SAISEU19-loan-risks.snappy.parquet’

     0K .......... .......... .......... .......... .......... 31% 25.6M 0s
    50K .......... .......... .......... .......... .......... 62% 38.3M 0s
   100K .......... .......... .......... .......... .......... 93% 12.3M 0s
   150K ..........                                            100%  125M=0.007s

2021-11-20 00:21:05 (21.7 MB/s) - ‘Data/delta_demo/loans/SAISEU19-loan-risks.snappy.parquet’ saved [164631/164631]



In [34]:
import boto3
import s3fs
import sagemaker

# Retrieve the default bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
prefix = "sagemaker-delta"
role = sagemaker.get_execution_role()
print(region)
print(bucket)

us-east-1
sagemaker-us-east-1-215839950614


In [55]:
import os
import shutil
from pyspark.sql.functions import * 


# delta_path = "Data/delta_demo/loans_delta/"
delta_path = "s3a://{}/{}/loans-delta".format(bucket, prefix)

# Delete a new delta table with the parquet file
if os.path.exists(delta_path):
    print("Deleting path " + delta_path)
    shutil.rmtree(delta_path)

# Create a new delta table with the parquet file
spark.read.format("parquet").load("Data/delta_demo/loans") \
  .write.format("delta").save(delta_path)
print("Created a Delta table at " + delta_path)

Py4JJavaError: An error occurred while calling o238.save.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2595)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3269)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:456)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:391)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:147)
	at org.apache.spark.sql.execu

In [41]:
# Create a view on the table called loans_delta
spark.read.format("delta").load(delta_path).createOrReplaceTempView("loans_delta")
print("Defined view 'loans_delta'")

spark.sql("select count(*) from loans_delta").show()

Defined view 'loans_delta'
+--------+
|count(1)|
+--------+
|   14705|
+--------+



In [42]:
loans_df = spark.sql("select * from loans_delta").toPandas()

In [43]:
loans_df.head()

,loan_id,funded_amnt,paid_amnt,addr_state
0,0,1000,182.22,CA
1,1,1000,361.19,WA
2,2,1000,176.26,TX
3,3,1000,1000.00,OK
4,4,1000,249.98,PA
